# Task 4
This serves as a template which will guide you through the implementation of this task. It is advised to first read the whole template and get a sense of the overall structure of the code before trying to fill in any of the TODO gaps.
This is the jupyter notebook version of the template. For the python file version, please refer to the file `template_solution.py`.

First, we import necessary libraries:

In [12]:
import numpy as np
import pandas as pd
import torch
from datasets import Dataset, DatasetDict
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    EarlyStoppingCallback,
    Trainer,
    TrainingArguments,
)
# Add any other imports you need here

Depending on your approach, you might need to adapt the structure of this template or parts not marked by TODOs.
It is not necessary to completely follow this template. Feel free to add more code and delete any parts that are not required.

In [13]:
def clear_cache():
    if torch.backends.mps.is_available():
        torch.mps.empty_cache()

In [19]:
# DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 128  # TODO: Set the batch size according to both training performance and available memory
NUM_EPOCHS = 10  # TODO: Set the number of epochs

# Load test data
test_val = pd.read_csv("test_no_score.csv")
test_val["text"] = test_val["title"] + " " + test_val["sentence"]
test_val = test_val[["text"]]


# Load train data
train_val = pd.read_csv("train.csv")

train_val["text"] = train_val["title"] + " " + train_val["sentence"]
train_val["label"] = train_val["score"]
train_val = train_val[["text", "label"]]

# Load datasets
train_dataset = Dataset.from_pandas(train_val)
test_dataset = Dataset.from_pandas(test_val)

train_test_split = train_dataset.train_test_split(test_size=0.1)
dataset = DatasetDict(
    {"train": train_test_split["train"], "validation": train_test_split["test"]}
)
test_dataset = DatasetDict({"test": test_dataset})


print(dataset["train"][0])

{'text': "Why I Bought This CD Forgotten Dreams by David Pietro was such a strong CD that I've ordered this latest CD without reading any reviews. I know I'll love it!", 'label': 9.377984268162235}


In [21]:
# Tokenize
tokenizer = AutoTokenizer.from_pretrained("roberta-base")


def tokenizer_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_dataset = dataset.map(tokenizer_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenizer_function)
print(tokenized_dataset["train"][0])

model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=1)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
    weight_decay=0.01,
    logging_steps=10,
    logging_first_step=True,
    gradient_accumulation_steps=4,
    load_best_model_at_end=True,
)


def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.squeeze()
    mse = mean_squared_error(labels, preds)
    return {"mse": mse}


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

Map:   0%|          | 0/11250 [00:00<?, ? examples/s]

Map:   0%|          | 0/1250 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

{'text': "Why I Bought This CD Forgotten Dreams by David Pietro was such a strong CD that I've ordered this latest CD without reading any reviews. I know I'll love it!", 'label': 9.377984268162235, 'input_ids': [0, 7608, 38, 45192, 152, 7522, 43799, 26843, 30, 871, 20964, 1001, 21, 215, 10, 670, 7522, 14, 38, 348, 2740, 42, 665, 7522, 396, 2600, 143, 6173, 4, 38, 216, 38, 581, 657, 24, 328, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
trainer.train(resume_from_checkpoint=True)

model.save_pretrained("./saved_model")
tokenizer.save_pretrained("./saved_model")

  0%|          | 0/7030 [00:00<?, ?it/s]

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 44.9809, 'learning_rate': 1.999715504978663e-05, 'epoch': 0.0}
{'loss': 29.3147, 'learning_rate': 1.997155049786629e-05, 'epoch': 0.01}
{'loss': 16.1213, 'learning_rate': 1.9943100995732575e-05, 'epoch': 0.03}
{'loss': 11.4306, 'learning_rate': 1.9914651493598865e-05, 'epoch': 0.04}
{'loss': 10.072, 'learning_rate': 1.9886201991465152e-05, 'epoch': 0.06}
{'loss': 10.9514, 'learning_rate': 1.985775248933144e-05, 'epoch': 0.07}
{'loss': 10.5677, 'learning_rate': 1.9829302987197725e-05, 'epoch': 0.09}
{'loss': 7.5098, 'learning_rate': 1.9800853485064012e-05, 'epoch': 0.1}
{'loss': 4.3041, 'learning_rate': 1.97724039829303e-05, 'epoch': 0.11}
{'loss': 4.5812, 'learning_rate': 1.974395448079659e-05, 'epoch': 0.13}
{'loss': 4.2453, 'learning_rate': 1.9715504978662876e-05, 'epoch': 0.14}
{'loss': 3.912, 'learning_rate': 1.9687055476529162e-05, 'epoch': 0.16}
{'loss': 3.2101, 'learning_rate': 1.965860597439545e-05, 'epoch': 0.17}
{'loss': 3.8125, 'learning_rate': 1.9630156472261736e-0

  0%|          | 0/313 [00:00<?, ?it/s]

{'eval_loss': 0.9705937504768372, 'eval_mse': 0.9705937504768372, 'eval_runtime': 6.2099, 'eval_samples_per_second': 201.292, 'eval_steps_per_second': 50.404, 'epoch': 1.0}
{'loss': 1.0648, 'learning_rate': 1.7980085348506404e-05, 'epoch': 1.01}
{'loss': 1.3022, 'learning_rate': 1.795163584637269e-05, 'epoch': 1.02}
{'loss': 0.8315, 'learning_rate': 1.7923186344238977e-05, 'epoch': 1.04}
{'loss': 0.7366, 'learning_rate': 1.7894736842105264e-05, 'epoch': 1.05}
{'loss': 0.733, 'learning_rate': 1.7866287339971554e-05, 'epoch': 1.07}
{'loss': 0.9304, 'learning_rate': 1.783783783783784e-05, 'epoch': 1.08}
{'loss': 0.7479, 'learning_rate': 1.7809388335704127e-05, 'epoch': 1.09}
{'loss': 0.7796, 'learning_rate': 1.7780938833570414e-05, 'epoch': 1.11}
{'loss': 0.748, 'learning_rate': 1.77524893314367e-05, 'epoch': 1.12}
{'loss': 0.8372, 'learning_rate': 1.7724039829302988e-05, 'epoch': 1.14}
{'loss': 0.8044, 'learning_rate': 1.7695590327169278e-05, 'epoch': 1.15}
{'loss': 0.7121, 'learning_rat

...

  0%|          | 0/313 [00:00<?, ?it/s]

{'eval_loss': 0.6002613306045532, 'eval_mse': 0.6002613306045532, 'eval_runtime': 6.048, 'eval_samples_per_second': 206.681, 'eval_steps_per_second': 51.753, 'epoch': 9.0}
{'loss': 0.108, 'learning_rate': 1.9914651493598865e-06, 'epoch': 9.0}
{'loss': 0.0836, 'learning_rate': 1.9630156472261737e-06, 'epoch': 9.02}
{'loss': 0.0768, 'learning_rate': 1.9345661450924613e-06, 'epoch': 9.03}
{'loss': 0.0833, 'learning_rate': 1.9061166429587482e-06, 'epoch': 9.04}
{'loss': 0.0863, 'learning_rate': 1.8776671408250358e-06, 'epoch': 9.06}
{'loss': 0.0815, 'learning_rate': 1.849217638691323e-06, 'epoch': 9.07}
{'loss': 0.0875, 'learning_rate': 1.8207681365576105e-06, 'epoch': 9.09}
{'loss': 0.0768, 'learning_rate': 1.7923186344238977e-06, 'epoch': 9.1}
{'loss': 0.0844, 'learning_rate': 1.7638691322901852e-06, 'epoch': 9.11}
{'loss': 0.0809, 'learning_rate': 1.7354196301564724e-06, 'epoch': 9.13}
{'loss': 0.1419, 'learning_rate': 1.7069701280227595e-06, 'epoch': 9.14}
{'loss': 0.1005, 'learning_ra

  0%|          | 0/313 [00:00<?, ?it/s]

{'eval_loss': 0.6054350137710571, 'eval_mse': 0.6054350137710571, 'eval_runtime': 6.0903, 'eval_samples_per_second': 205.243, 'eval_steps_per_second': 51.393, 'epoch': 10.0}
{'train_runtime': 1827.4649, 'train_samples_per_second': 61.561, 'train_steps_per_second': 3.847, 'train_loss': 0.5604167800794115, 'epoch': 10.0}


('./saved_model/tokenizer_config.json',
 './saved_model/special_tokens_map.json',
 './saved_model/vocab.json',
 './saved_model/merges.txt',
 './saved_model/added_tokens.json',
 './saved_model/tokenizer.json')

In [29]:
# Do predictions
predictions = trainer.predict(tokenized_test_dataset["test"])
pred_scores = predictions.predictions.squeeze()

with open("results.txt", "w") as f:
    for val in pred_scores:
        f.write(f"{val}\n")

print("Saved results")

  0%|          | 0/250 [00:00<?, ?it/s]

Saved results


In [ ]:
# model.eval()
# with torch.no_grad():
#     results = []
#     for batch in tqdm(test_loader, total=len(test_loader)):
#         batch = batch.to(DEVICE)

#         # TODO: Set up evaluation loop

#     with open("result.txt", "w") as f:
#         for val in np.concatenate(results):
#             f.write(f"{val}\n")

NameError: name 'torch' is not defined